## Imports and Setup

In [1]:
import json
import geopandas as gpd
import pandas as pd
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

print("📦 Libraries imported successfully")
print(f"   📍 GeoPandas version: {gpd.__version__}")
print(f"   📊 Pandas version: {pd.__version__}")

📦 Libraries imported successfully
   📍 GeoPandas version: 1.1.1
   📊 Pandas version: 2.2.3


## Load GeoJSON File

In [2]:
# Path to your GeoJSON file
geojson_path = "../census tract geofiles/manhattan_census_tracts.geojson"

# Load the GeoJSON file
print("📖 Loading census tract geometries...")

gdf = gpd.read_file(geojson_path)

print(f"✅ Loaded {len(gdf)} census tracts")
print(f"   📊 Columns: {list(gdf.columns)}")
print(f"   🗺️  Original CRS: {gdf.crs}")

# Check for GEOID column
if 'GEOID' in gdf.columns:
    print(f"✅ GEOID column found")
    print(f"   📝 Sample GEOIDs: {gdf['GEOID'].head(3).tolist()}")
else:
    print(f"❌ GEOID column not found! Available: {list(gdf.columns)}")

📖 Loading census tract geometries...
✅ Loaded 310 census tracts
   📊 Columns: ['GEOID', 'CTLabel', 'NTAName', 'NTA2020', 'CDTA2020', 'CDTANAME', 'BoroName', 'CT2020', 'BoroCT2020', 'geometry']
   🗺️  Original CRS: EPSG:4326
✅ GEOID column found
   📝 Sample GEOIDs: [36061000100, 36061001401, 36061001402]


## Convert to WGS84 and Compute Centroids

In [3]:
# Convert to WGS84 (EPSG:4326) for Google Maps compatibility
if gdf.crs != "EPSG:4326":
    print(f"🔄 Converting from {gdf.crs} to EPSG:4326 (WGS84)")
    gdf = gdf.to_crs(epsg=4326)
else:
    print(f"✅ Already in WGS84 coordinate system")

# Compute centroids
print("🎯 Computing tract centroids...")

centroids = gdf.geometry.centroid
latitudes = centroids.y.round(6)  # 6 decimal places = ~11cm precision
longitudes = centroids.x.round(6)

# Create centroids DataFrame
centroids_df = pd.DataFrame({
    'GEOID': gdf['GEOID'].astype(str),
    'latitude': latitudes,
    'longitude': longitudes
})

print(f"✅ Computed {len(centroids_df)} centroids")
print(f"   📍 Lat range: {latitudes.min():.6f} to {latitudes.max():.6f}")
print(f"   📍 Lng range: {longitudes.min():.6f} to {longitudes.max():.6f}")

✅ Already in WGS84 coordinate system
🎯 Computing tract centroids...
✅ Computed 310 centroids
   📍 Lat range: 40.688954 to 40.875966
   📍 Lng range: -74.042340 to -73.910374


## Validate Coordinates

In [4]:
# Validate coordinates are within NYC bounds
print("🔍 Validating coordinates...")

nyc_bounds = {
    'lat_min': 40.4, 'lat_max': 40.9,
    'lng_min': -74.3, 'lng_max': -73.7
}

valid_coords = (
    (centroids_df['latitude'] >= nyc_bounds['lat_min']) &
    (centroids_df['latitude'] <= nyc_bounds['lat_max']) &
    (centroids_df['longitude'] >= nyc_bounds['lng_min']) &
    (centroids_df['longitude'] <= nyc_bounds['lng_max'])
)

valid_count = valid_coords.sum()
invalid_count = (~valid_coords).sum()

print(f"✅ Valid coordinates: {valid_count} / {len(centroids_df)}")
if invalid_count > 0:
    print(f"⚠️  Invalid coordinates: {invalid_count}")
else:
    print(f"🎉 All coordinates within expected NYC bounds!")

# Display sample data
print(f"\n📋 Sample Data:")
print(centroids_df.head())

🔍 Validating coordinates...
✅ Valid coordinates: 310 / 310
🎉 All coordinates within expected NYC bounds!

📋 Sample Data:
         GEOID   latitude  longitude
0  36061000100  40.695673 -74.042340
1  36061001401  40.715160 -73.987332
2  36061001402  40.717513 -73.985882
3  36061001800  40.719046 -73.990842
4  36061002201  40.719116 -73.981844


## Export as JSON

In [5]:
# Create JSON structure for Next.js TypeScript
print("📤 Creating JSON export...")

tract_centroids = {}

for _, row in centroids_df.iterrows():
    tract_centroids[row['GEOID']] = {
        'lat': float(row['latitude']),
        'lng': float(row['longitude'])
    }

print(f"✅ Created JSON structure with {len(tract_centroids)} tracts")

# Export to JSON file
output_path = "tract_centroids.json"

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(tract_centroids, f, indent=2, ensure_ascii=False)

file_size = Path(output_path).stat().st_size
print(f"\n✅ JSON exported successfully!")
print(f"   📁 File: {output_path}")
print(f"   📏 Size: {file_size:,} bytes")
print(f"   📊 Contains: {len(tract_centroids)} tract centroids")

📤 Creating JSON export...
✅ Created JSON structure with 310 tracts

✅ JSON exported successfully!
   📁 File: tract_centroids.json
   📏 Size: 22,262 bytes
   📊 Contains: 310 tract centroids


## Test and Usage Examples

In [6]:
# Test the exported JSON
with open("tract_centroids.json", 'r') as f:
    loaded_data = json.load(f)

print(f"🔍 JSON loads correctly: {len(loaded_data)} entries")

# Show sample entries
sample_geoids = list(loaded_data.keys())[:3]
print(f"\n📝 Sample entries:")
for geoid in sample_geoids:
    coords = loaded_data[geoid]
    print(f"   {geoid}: lat={coords['lat']:.6f}, lng={coords['lng']:.6f}")

print(f"\n🚀 Next.js TypeScript Usage:")
print(f"```typescript")
print(f"import tractCentroids from './tract_centroids.json';")
print(f"")
print(f"// Get coordinates")
print(f"const coords = tractCentroids['{sample_geoids[0]}'];")
print(f"")
print(f"// Google Maps Static API URL")
print(f"const mapUrl = `https://maps.googleapis.com/maps/api/staticmap?")
print(f"center=${{coords.lat}},${{coords.lng}}&zoom=16&size=400x400&")
print(f"markers=color:red|${{coords.lat}},${{coords.lng}}&")
print(f"key=${{process.env.NEXT_PUBLIC_GOOGLE_MAPS_API_KEY}}`;")
print(f"```")

print(f"\n🎯 Ready for integration with your Brickwyze frontend!")

🔍 JSON loads correctly: 310 entries

📝 Sample entries:
   36061000100: lat=40.695673, lng=-74.042340
   36061001401: lat=40.715160, lng=-73.987332
   36061001402: lat=40.717513, lng=-73.985882

🚀 Next.js TypeScript Usage:
```typescript
import tractCentroids from './tract_centroids.json';

// Get coordinates
const coords = tractCentroids['36061000100'];

// Google Maps Static API URL
const mapUrl = `https://maps.googleapis.com/maps/api/staticmap?
center=${coords.lat},${coords.lng}&zoom=16&size=400x400&
markers=color:red|${coords.lat},${coords.lng}&
key=${process.env.NEXT_PUBLIC_GOOGLE_MAPS_API_KEY}`;
```

🎯 Ready for integration with your Brickwyze frontend!
